<a href="https://colab.research.google.com/github/ykitaguchi77/CorneAI/blob/main/50_knock_csv_manipulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**「前原の100本ノック」クラス枚数確認**

In [1]:
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [2]:
import pandas as pd

# 100問_正解リスト.csvファイルからデータフレームを作成する
ans_df = pd.read_csv('/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の100問/100問_正解リスト.csv')

# 100問ノック_スマホと細隙灯区分.csvファイルからデータフレームを作成する
knock_df = pd.read_csv('/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の100問/100問ノック_スマホと細隙灯区分.csv')

In [ ]:
ans_df

In [ ]:
knock_df

In [ ]:
import pandas as pd

# "device"の行をdf_answerに追加
ans_df['device'] = ""

# "slit"と"smartphone"の列を取得
slit = knock_df["slit"].tolist()
smartphone = knock_df["smartphone"].tolist()

# "device"列に"slit"と"smartphone"を入力
for i in range(1,101):
    if i in slit:
        ans_df.loc[ans_df["number"]==i, "device"] = "slit"
    elif i in smartphone:
        ans_df.loc[ans_df["number"]==i, "device"] = "smartphone"

# インデックスをリセット
ans_df = ans_df.reset_index(drop=True)
ans_df

In [ ]:
smartphone_df = ans_df[ans_df['device'] == 'smartphone']
smartphone_classification_counts = smartphone_df['classification'].value_counts()
print("前原100本ノック_スマホ画像の各クラス枚数一覧")
print(smartphone_classification_counts)

前原100本ノック_スマホ画像の各クラス枚数一覧
正常        7
沈着        7
水晶体混濁     6
感染        6
腫瘍        6
水疱性角膜症    5
非感染       5
緑内障発作     4
瘢痕        4
Name: classification, dtype: int64


In [ ]:
slit_df = ans_df[ans_df['device'] == 'slit']
slit_classification_counts = slit_df['classification'].value_counts()
print("前原100本ノック_スリット画像の各クラス枚数一覧")
print(slit_classification_counts)

前原100本ノック_スリット画像の各クラス枚数一覧
正常        7
沈着        7
腫瘍        6
感染        6
水晶体混濁     6
水疱性角膜症    5
非感染       5
瘢痕        4
緑内障発作     4
Name: classification, dtype: int64


In [ ]:
ans_df


In [ ]:
ans_df[ans_df["number"]==1]

,disease,classification,number,device
0,翼状片,腫瘍,1,slit


In [ ]:
ans_df

In [ ]:
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 150)

In [ ]:
ans_df = ans_df.replace("腫瘍", "tumor")
ans_df = ans_df.replace("瘢痕", "scar")
ans_df = ans_df.replace("水晶体混濁", "lens-opacity")
ans_df = ans_df.replace("緑内障発作", "APAC")
ans_df = ans_df.replace("感染", "infection")
ans_df = ans_df.replace("非感染", "non-infection")
ans_df = ans_df.replace("水疱性角膜症", "bullous")
ans_df = ans_df.replace("沈着", "deposit")
ans_df = ans_df.replace("正常", "normal")
ans_df


In [97]:
ans_df.to_csv("/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の100問/100問_正解リスト_2.csv")

In [96]:
ans_df.columns

Index(['Unnamed: 0', 'disease', 'classification', 'number', 'device'], dtype='object')

#**Computer vs Smartphone**

In [3]:
data_dir = "/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の100問/PC_vs_smartphone.csv"


data_df = pd.read_csv(data_dir, encoding='cp932')
data_df
data_df.columns = ['ファイル名', 'CorneAI_PC', 'normal', 'infection', 'non-infection', 'scar', 'deposit', 'bullous',
       'tumor', 'lens-opacity', 'APAC', 'Unnamed: 11', 'CorneAI_ios',
       'normal', 'infection', 'non-infection', 'scar', 'deposit', 'bullous',
       'tumor', 'lens-opacity', 'APAC']


In [4]:
# add answer and device information to data_dir
ans_df = pd.read_csv("/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の100問/100問_正解リスト_2.csv")
data_df["device"] = ans_df["device"]
data_df["answer"] = ans_df["classification"]

In [ ]:
data_df

In [6]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 150)


In [60]:
import random
import numpy as np

# top1の確率、各top3のindexを出力する

def top3(data_df, row):
    # computerとsmartphoneの判定の列を別々に取り出す
    computer_cols = data_df.iloc[row, 2:11].astype(float).fillna(0)
    smartphone_cols = data_df.iloc[row, 13:22].astype(float).fillna(0)

    # numpy.argsortを使って、各列の値が降順になるようにインデックスを並び替える
    computer_idx = np.argsort(-computer_cols)
    smartphone_idx = np.argsort(-smartphone_cols)

    # インデックスから列名を取得する
    computer_names = computer_cols.index[computer_idx]
    smartphone_names = smartphone_cols.index[smartphone_idx]

    # computerとsmartphoneのtop1の確率を取得する
    computer_top_prob = computer_cols[computer_names[0]]
    smartphone_top_prob = smartphone_cols[smartphone_names[0]]

    #answerとdeviceと追加
    device = data_df.loc[row, "device"]
    answer = data_df.loc[row, "answer"]

    # top3の列名とtop1の確率をリストとして返す
    return [computer_names[0], computer_top_prob, computer_names[1], computer_names[2], smartphone_names[0],  smartphone_top_prob/100, smartphone_names[1], smartphone_names[2], answer, device]
 
top3(data_df, 71)

['APAC',
 0.953017,
 'normal',
 'infection',
 'APAC',
 0.96,
 'lens-opacity',
 'non-infection',
 'APAC',
 'slit']

In [ ]:
top3_df = pd.DataFrame(index=[], columns=["computer_top1", "computer_top1_prob", "computer_top2", "computer_top3", "smartphone_top1", "smartphone_top1_prob", "smartphone_top2", "smartphone_top3", "answer", "device"])

for row in range(len(data_df)):
    top3_df.loc[row, ["computer_top1", "computer_top1_prob", "computer_top2", "computer_top3", "smartphone_top1", "smartphone_top1_prob", "smartphone_top2", "smartphone_top3", "answer", "device"]] = top3(data_df, row)
top3_df

In [11]:
# take_slit_judge_computer = top3_df.loc[(top3_df["device"] == "slit"), "computer_top1"].tolist()
# take_slit_judge_smartphone = top3_df.loc[(top3_df["device"] == "slit"), "smartphone_top1"].tolist()
# take_smartphone_judge_computer = top3_df.loc[(top3_df["device"] == "smartphone"), "computer_top1"].tolist()
# take_smartphone_judge_smartphone = top3_df.loc[(top3_df["device"] == "smartphone"), "smartphone_top1"].tolist()
# take_slit_answer = top3_df.loc[top3_df["device"] == "slit", "answer"].tolist()
# take_smartphone_answer = top3_df.loc[top3_df["device"] == "smartphone", "answer"].tolist()

In [12]:
# # calculate top1 accuracy
# computer_correct = sum(top3_df["computer_top1"] == top3_df["answer"])
# smartphone_correct = sum(top3_df["smartphone_top1"] == top3_df["answer"])
# correspond = sum(top3_df["smartphone_top1"] == top3_df["computer_top1"])


# print(f"computer_top1: {computer_correct}/100")
# print(f"smartphone_top1: {smartphone_correct}/100") 
# print(f"correspond: {correspond}/100")  

computer_top1: 71/100
smartphone_top1: 78/100
correspond: 73/100


In [13]:
# # calculate top3 accuracy
# computer_correct = top3_df[["computer_top1", "computer_top2", "computer_top3"]].eq(top3_df["answer"], axis=0).any(axis=1).sum()
# smartphone_correct = top3_df[["smartphone_top1", "smartphone_top2", "smartphone_top3"]].eq(top3_df["answer"], axis=0).any(axis=1).sum()
# smartphone_top1_in_computer_top3 = top3_df[["computer_top1", "computer_top2", "computer_top3"]].eq(top3_df["smartphone_top1"], axis=0).any(axis=1).sum()

# print(f"computer_top3: {computer_correct}/100")
# print(f"smartphone_top3: {smartphone_correct}/100") 
# print(f"smartphone_top1 in computer_top3: {smartphone_top1_in_computer_top3}/100")  

computer_top3: 81/100
smartphone_top3: 90/100
smartphone_top1 in computer_top3: 80/100


In [62]:
def get_matching_rows_count(text, df, col1, col2, devices):
    matching_rows = 0
    num_device_rows = 0
    for device in devices:
        # df[device_col] = deviceである行の数：
        device_rows = df[df["device"] == device]
        num_device_rows += len(device_rows)

        # df[device_col] = deviceである行のうち、df[col2]とdf[col1]が一致する行の数：
        matching_rows += df[df["device"] == device][col2].eq(df[col1], axis=0).any(axis=1).sum()

    percent = matching_rows * 100 / num_device_rows

    print(text)
    print(f"accuracy: {percent}% ({matching_rows}/{num_device_rows})")
    print("")

get_matching_rows_count("smartphone vs computer", top3_df, "smartphone_top1", ["computer_top1"], ["slit", "smartphone"])
get_matching_rows_count("computer accuracy", top3_df, "answer", ["computer_top1"], ["slit", "smartphone"])
get_matching_rows_count("smartphone accuracy", top3_df, "answer", ["smartphone_top1"], ["slit", "smartphone"])
print("")
get_matching_rows_count("smartphone vs computer_top3", top3_df, "smartphone_top1", ["computer_top1", "computer_top2", "computer_top3"], ["slit", "smartphone"])
get_matching_rows_count("computer_top3 accuracy", top3_df, "answer", ["computer_top1", "computer_top2", "computer_top3"], ["slit", "smartphone"])
get_matching_rows_count("smartphone_top3 accuracy", top3_df, "answer", ["smartphone_top1", "smartphone_top2", "smartphone_top3"], ["slit", "smartphone"])
print("")
get_matching_rows_count("taken by smartphone、smartphone vs computer", top3_df, "smartphone_top1", ["computer_top1"], ["smartphone"])
get_matching_rows_count("taken by slit、smartphone vs computer", top3_df, "smartphone_top1", ["computer_top1"], ["slit"])
get_matching_rows_count("taken by slit、computer accuracy", top3_df, "answer", ["computer_top1"], ["slit"])
get_matching_rows_count("taken by slit、smartphone accuracy", top3_df, "answer", ["smartphone_top1"], ["slit"])
get_matching_rows_count("taken by smartphone、computer accuracy", top3_df, "answer", ["computer_top1"], ["smartphone"])
get_matching_rows_count("taken by smartphone、smartphone accuracy", top3_df, "answer", ["smartphone_top1"], ["smartphone"])
print("")
get_matching_rows_count("taken by smartphone、smartphone vs computer_top3", top3_df, "smartphone_top1", ["computer_top1", "computer_top2", "computer_top3"], ["smartphone"])
get_matching_rows_count("taken by slit、smartphone vs computer_top3", top3_df, "smartphone_top1", ["computer_top1", "computer_top2", "computer_top3"], ["slit"])
get_matching_rows_count("taken by slit、computer_top3 accuracy", top3_df, "answer", ["computer_top1", "computer_top2", "computer_top3"], ["slit"])
get_matching_rows_count("taken by slit、smartphone_top3 accuracy", top3_df, "answer", ["smartphone_top1", "smartphone_top2", "smartphone_top3"], ["slit"])
get_matching_rows_count("taken by smartphone、computer_top3 accuracy", top3_df, "answer", ["computer_top1", "computer_top2", "computer_top3"], ["smartphone"])
get_matching_rows_count("taken by smartphone、smartphone_top3 accuracy", top3_df, "answer", ["smartphone_top1", "smartphone_top2", "smartphone_top3"], ["smartphone"])

smartphone vs computer
accuracy: 77.0% (77/100)

computer accuracy
accuracy: 75.0% (75/100)

smartphone accuracy
accuracy: 78.0% (78/100)


smartphone vs computer_top3
accuracy: 84.0% (84/100)

computer_top3 accuracy
accuracy: 85.0% (85/100)

smartphone_top3 accuracy
accuracy: 90.0% (90/100)


taken by smartphone、smartphone vs computer
accuracy: 76.0% (38/50)

taken by slit、smartphone vs computer
accuracy: 78.0% (39/50)

taken by slit、computer accuracy
accuracy: 78.0% (39/50)

taken by slit、smartphone accuracy
accuracy: 84.0% (42/50)

taken by smartphone、computer accuracy
accuracy: 72.0% (36/50)

taken by smartphone、smartphone accuracy
accuracy: 72.0% (36/50)


taken by smartphone、smartphone vs computer_top3
accuracy: 82.0% (41/50)

taken by slit、smartphone vs computer_top3
accuracy: 86.0% (43/50)

taken by slit、computer_top3 accuracy
accuracy: 86.0% (43/50)

taken by slit、smartphone_top3 accuracy
accuracy: 94.0% (47/50)

taken by smartphone、computer_top3 accuracy
accuracy: 84.0% (42/5

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

def heatmap(df_a, df_b, rownames, colnames):
    # 混同行列の作成
    confusion_matrix = pd.crosstab(df_a, df_b, rownames=rownames, colnames=colnames)
    # ヒートマップのプロット
    sns.heatmap(confusion_matrix, annot=True, cmap='Blues', fmt='g')
    plt.show()


# computer_top1 vs smartphone_top1
heatmap(top3_df["computer_top1"], top3_df["smartphone_top1"], ["computer_top1"], ["smatrphone_top1"])


In [ ]:
# computer_top1 vs answer
heatmap(top3_df["computer_top1"], top3_df["answer"], ["computer_top1"], ["groudtruth"])

In [ ]:
# computer_top1_taken_by_slit vs answer
heatmap(top3_df[top3_df["device"]=="slit"]["computer_top1"], top3_df[top3_df["device"]=="slit"]["answer"], ["computer_top1_taken_by_slit"], ["groudtruth"])

In [ ]:
# computer_top1_taken_by_smartphone vs answer
heatmap(top3_df[top3_df["device"]=="smartphone"]["computer_top1"], top3_df[top3_df["device"]=="smartphone"]["answer"], ["computer_top1_taken_by_smartphone"],  ["groudtruth"])

In [ ]:
# smartphone_top1 vs answer
heatmap(top3_df["smartphone_top1"], top3_df["answer"], ["smartphone_top1"], ["groudtruth"])

In [ ]:
# smartphone_top1_taken_by_slit vs answer
heatmap(top3_df[top3_df["device"]=="slit"]["smartphone_top1"], top3_df[top3_df["device"]=="slit"]["answer"], ["smartphone_top1_taken_by_slit"], ["groudtruth"])

In [ ]:
# smartphone_top1_taken_by_smartphone vs answer
heatmap(top3_df[top3_df["device"]=="smartphone"]["smartphone_top1"], top3_df[top3_df["device"]=="smartphone"]["answer"], ["smartphone_top1_taken_by_smartphone"],  ["groudtruth"])